In [59]:
import pandas as pd
import requests

In [107]:
lat = -33.8034688
lng = 151.1882752
r = requests.get(f"http://localhost:8000/nearby?lat={lat}&lng={lng}")

In [106]:
data = r.json()
data

{'summary': {'House,2.0': {'medianPrice_L12M': 2580000.0,
   'TranxLTM_L12M': 1,
   'medianPrice_L12M_prev': -1.0,
   'TranxLTM_L12M_prev': -1.0},
  'House,3.0': {'medianPrice_L12M': 2757500.0,
   'TranxLTM_L12M': 8,
   'medianPrice_L12M_prev': 3300000.0,
   'TranxLTM_L12M_prev': 7.0},
  'House,4.0': {'medianPrice_L12M': 3820000.0,
   'TranxLTM_L12M': 15,
   'medianPrice_L12M_prev': 3700000.0,
   'TranxLTM_L12M_prev': 9.0},
  'House,5.0': {'medianPrice_L12M': 4130500.0,
   'TranxLTM_L12M': 4,
   'medianPrice_L12M_prev': 4344000.0,
   'TranxLTM_L12M_prev': 2.0}}}

In [54]:
df = pd.DataFrame(data)
df.head()

,property_id,beds,baths,parking,propertyType,propertyTypeFormatted,isRural,landSize,landUnit,isRetirement,...,suburb,state,postcode,lat,lng,price,tagText,tagClassName,soldDate,location
0,2018898832,4.0,2.0,1.0,House,House,False,695.0,m²,False,...,ARTARMON,NSW,2064,-33.805878,151.188049,4380000,Sold prior to auction 29 Nov 2023,is-sold,2023-11-29,"{'type': 'Point', 'coordinates': [151.188049, ..."
1,2018547783,4.0,4.0,1.0,House,House,False,967.0,m²,False,...,ARTARMON,NSW,2064,-33.806206,151.185410,4850000,Sold prior to auction 16 Jun 2023,is-sold,2023-06-16,"{'type': 'Point', 'coordinates': [151.18541, -..."
2,2018784277,3.0,2.0,1.0,House,House,False,0.0,m²,False,...,ARTARMON,NSW,2064,-33.807243,151.185410,2800000,Sold by private treaty 10 Oct 2023,is-sold,2023-10-10,"{'type': 'Point', 'coordinates': [151.18541, -..."
3,2013407012,3.0,2.0,2.0,House,House,False,695.0,m²,False,...,ARTARMON,NSW,2064,-33.807423,151.190109,2660000,Sold at auction 18 Mar 2017,is-sold,2017-03-18,"{'type': 'Point', 'coordinates': [151.190109, ..."
4,2017691857,3.0,2.0,2.0,House,House,False,695.0,m²,False,...,ARTARMON,NSW,2064,-33.807476,151.190100,2660000,Sold at auction 18 Mar 2017,is-sold,2017-03-18,"{'type': 'Point', 'coordinates': [151.1901, -3..."


In [57]:
def nearby_summary(data: dict):
  if len(data) == 0:
    return {}
  df = pd.DataFrame(data)
  house = ['House', 'SemiDetached', 'Townhouse']
  unit = ['ApartmentUnitFlat']
  df['house_or_unit'] = df['propertyType'].apply(lambda x: 'House' if x in house else 'Unit' if x in unit else 'Other')
  df['soldDate'] = pd.to_datetime(df['soldDate'])
  maxDate = df['soldDate'].max()
  L12MDate = df['soldDate'].max() - pd.DateOffset(months=12)
  L12MDate_prev = df['soldDate'].max() - pd.DateOffset(months=24)
  df_L12M = df[(df['soldDate'] >= L12MDate) & (df['soldDate'] <= maxDate)]
  df_L12M_prev = df[(df['soldDate'] >= L12MDate_prev) & (df['soldDate'] <= L12MDate)]
  grouped_df_L12M = df_L12M.groupby(['propertyTypeFormatted', 'beds']).agg(
    medianPrice=('price', 'median'),
    TranxLTM=('price', 'count')
  )
  grouped_df_L12M_prev = df_L12M_prev.groupby(['propertyTypeFormatted', 'beds']).agg(
    medianPrice=('price', 'median'),
    TranxLTM=('price', 'count')
  )
  joined_df = grouped_df_L12M.join(grouped_df_L12M_prev, on=['propertyTypeFormatted', 'beds'], lsuffix='_L12M', rsuffix='_L12M_prev')
  joined_dict = joined_df.to_dict(orient='index')
  return joined_dict
  

In [58]:
nearby_summary(data)

{('House', 2.0): {'medianPrice_L12M': 2580000.0,
  'TranxLTM_L12M': 1,
  'medianPrice_L12M_prev': nan,
  'TranxLTM_L12M_prev': nan},
 ('House', 3.0): {'medianPrice_L12M': 2757500.0,
  'TranxLTM_L12M': 8,
  'medianPrice_L12M_prev': 3300000.0,
  'TranxLTM_L12M_prev': 7.0},
 ('House', 4.0): {'medianPrice_L12M': 3820000.0,
  'TranxLTM_L12M': 15,
  'medianPrice_L12M_prev': 3700000.0,
  'TranxLTM_L12M_prev': 9.0},
 ('House', 5.0): {'medianPrice_L12M': 4130500.0,
  'TranxLTM_L12M': 4,
  'medianPrice_L12M_prev': 4344000.0,
  'TranxLTM_L12M_prev': 2.0}}